# Model: Pretrained Inception Network

## Preamble

In [1]:
from tensorflow import keras
import PIL
import numpy
import pandas
import itertools

## Data Preparation

In [2]:
movies = catalog.load("movies")

2021-03-15 22:00:13,166 - kedro.io.data_catalog - INFO - Loading data from `movies` (ParquetDataSet)...


**exclude test set**

In [3]:
movies_test = catalog.load("movies_test")

2021-03-15 22:00:13,368 - kedro.io.data_catalog - INFO - Loading data from `movies_test` (CSVDataSet)...


In [4]:
movies_test

,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,,,
17335,Obsessed,/g7kv7RXzqCSMmiYmV0gVLpxB0YH.jpg,['Drama' 'Thriller'],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
757,Murder Most Foul,/qTCKB5rxYzgdWUUH4UMgpRVH3GJ.jpg,['Comedy' 'Crime' 'Drama' 'Mystery' 'Thriller'],0,0,0,1,1,0,1,...,0,0,0,1,0,0,0,1,0,0
11886,Robin Hood,/x9AvkYek0bGdxQSZ8W3lAjGrREm.jpg,['Animation' 'Family'],0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2115,Light of Day,/hFXnASDyZd3i9TOuBPUtJRvfZsK.jpg,['Music' 'Drama'],0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1915,"Manta, Manta",/cpc3fbClSG46azA1dNW0ilKvIgj.jpg,['Comedy' 'Action'],1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,All Day Every Day,NaN,[],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18002,"Love, Honour and Obey",/n4mq89w2Dgn2oDnYGtx5qnSmfTA.jpg,['Action' 'Comedy' 'Thriller' 'Crime'],1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1830,Lord of War,/xEdwdFZRwrNAvTDx0fAV3MAInaA.jpg,['Crime' 'Drama' 'Thriller'],0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
movies_train = movies.loc[movies.index.difference(movies_test.index)]

In [6]:
movies_train

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genre_names
id,,,,,,,,,,,,,,,,,,,,,
2,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,,Ariel,False,6.8,128,NaN,None,None,None,"[Drama, Crime, Comedy]"
3,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,,Shadows in Paradise,False,7.2,125,NaN,None,None,None,"[Drama, Comedy]"
5,False,/u0zMKKpEdDWpOKmFW2sLbKKICJH.jpg,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.7,1901,NaN,None,None,None,"[Crime, Comedy]"
6,False,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,NaN,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",...,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.5,184,NaN,None,None,None,"[Action, Thriller, Crime]"
8,False,None,NaN,42000,"[{'id': 99, 'name': 'Documentary'}]",http://www.lifeinloops.com,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,...,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.3,13,NaN,None,None,None,[Documentary]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,False,/gpEJ2fR3Oy7hAMl8wAtv6Nd40x3.jpg,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",,tt1131734,en,Jennifer's Body,"Jennifer, a gorgeous, seductive cheerleader wh...",...,She's evil... and not just high school evil.,Jennifer's Body,False,5.7,2451,NaN,None,None,None,"[Comedy, Horror]"
19996,False,None,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",,tt0932661,en,Brother's War,The story is set in the latter days of World W...,...,,Brother's War,False,3.9,14,NaN,None,None,None,"[Action, Drama]"
19997,False,/TNfPn4Fwi0Fqx4BsjxOL3Tmscw.jpg,NaN,1250000,"[{'id': 53, 'name': 'Thriller'}]",,tt0046126,en,Niagara,"Rose Loomis and her older, gloomier husband, G...",...,A raging torrent of emotion that even nature c...,Niagara,False,6.8,214,NaN,None,None,None,[Thriller]


## Image Classifier

In [7]:
class ImageFileClassifier:
    
    def __init__(self, img_dir, min_probability=0.5):
        self.model = keras.applications.inception_v3.InceptionV3()
        self.input_dim = (299,299)
        self.min_probability = min_probability
        self.img_dir = img_dir
        
    def classify(self, img_path, as_list=True):
        """
        Classify an image.
        
        img_path: path pointing to an image file
        """
        if img_path is None:
            return []
        img_path = f"{self.img_dir}{img_path}"
        try:
            with PIL.Image.open(img_path) as img:
                img = img.resize(self.input_dim)
                img_a = keras.preprocessing.image.img_to_array(img)
                img_a = keras.applications.inception_v3.preprocess_input(img_a)
                predictions = self.model.predict(
                    numpy.array([img_a])
                )
                predictions_decoded = keras.applications.imagenet_utils.decode_predictions(predictions)[0]
                predictions = pandas.DataFrame(
                    predictions_decoded,
                    columns=["class_label", "class_name", "class_probability"]
                )
                # result
                if self.min_probability:
                    predictions = predictions[predictions["class_probability"] >= self.min_probability]
            return list(predictions["class_name"])
        except Exception as ex:
            #print(ex)
            return []


In [8]:
classifier = ImageFileClassifier(
    img_dir="../data/01_raw/posters",
    min_probability=0.1
)

In [9]:
classifier.classify("/1Crro6hN91lXc1sWXkU52tfjpAs.jpg")

['revolver']

In [10]:
from tqdm.auto import tqdm
tqdm.pandas()

In [11]:
%%time
movies["objects"] = movies["poster_path"].apply(classifier.classify)

2021-03-15 22:00:36,372 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (None, 299, 299, 1).
2021-03-15 22:00:39,189 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (None, 299, 299, 1).
2021-03-15 22:00:53,171 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shap

In [12]:
label_counts = pandas.Series(
    itertools.chain.from_iterable(
        movies[~movies["objects"].isna()]["objects"]
    )
).value_counts()

In [13]:
label_counts[:60]

comic_book            4037
book_jacket           3373
web_site              1170
stage                  346
suit                   234
maillot                189
bow_tie                182
jersey                 165
wig                    162
gown                   143
groom                  132
sunglasses             129
sunglass               127
hair_spray             123
brassiere              121
mask                   118
packet                 114
envelope               108
bikini                 107
jean                   100
menu                    85
miniskirt               84
theater_curtain         84
spotlight               82
military_uniform        81
cowboy_hat              79
television              78
rifle                   76
bobsled                 71
candle                  67
swimming_trunks         63
fountain                60
Band_Aid                59
sweatshirt              57
bathing_cap             56
tray                    56
academic_gown           56
p

In [14]:
label_counts[60:120]

neck_brace          36
assault_rifle       35
crash_helmet        34
bubble              34
scoreboard          31
prison              31
cloak               31
umbrella            31
feather_boa         31
parachute           31
volcano             30
racer               30
violin              29
seashore            29
fur_coat            27
park_bench          26
paddle              26
pick                26
tub                 26
hoopskirt           26
trench_coat         24
barbell             24
brass               24
loupe               23
sarong              23
harmonica           22
jack-o'-lantern     22
bow                 22
sleeping_bag        22
slot                22
scuba_diver         21
snowmobile          21
punching_bag        20
stole               20
sunscreen           20
syringe             20
bulletproof_vest    20
cab                 20
bearskin            19
limousine           19
football_helmet     19
street_sign         19
stethoscope         18
shower_cap 

In [15]:
#movies.to_parquet("../data/04_feature/movies_objects.parquet")

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

In [17]:
mlb = MultiLabelBinarizer()

In [18]:
mlb.fit(movies["objects"])

MultiLabelBinarizer()

In [19]:
mlb.classes_

array(['Afghan_hound', 'African_chameleon', 'African_crocodile',
       'African_elephant', 'African_hunting_dog',
       'American_Staffordshire_terrier', 'American_alligator',
       'Appenzeller', 'Arabian_camel', 'Australian_terrier', 'Band_Aid',
       'Border_collie', 'Border_terrier', 'Boston_bull',
       'Brittany_spaniel', 'CD_player', 'Chesapeake_Bay_retriever',
       'Chihuahua', 'Christmas_stocking', 'Crock_Pot', 'Dungeness_crab',
       'Egyptian_cat', 'Eskimo_dog', 'French_bulldog', 'French_horn',
       'French_loaf', 'German_shepherd', 'Granny_Smith',
       'Greater_Swiss_Mountain_dog', 'Indian_cobra', 'Indian_elephant',
       'Irish_setter', 'Irish_water_spaniel', 'Labrador_retriever',
       'Leonberg', 'Lhasa', 'Loafer', 'Model_T', 'Newfoundland',
       'Norfolk_terrier', 'Norwich_terrier', 'Old_English_sheepdog',
       'Pekinese', 'Persian_cat', 'Petri_dish', 'Polaroid_camera',
       'Pomeranian', 'Rhodesian_ridgeback', 'Rottweiler', 'Saint_Bernard',
       '

In [20]:
objects = pandas.DataFrame(
    mlb.transform(movies["objects"]),
    columns=mlb.classes_,
    index=movies.index
)
objects

,Afghan_hound,African_chameleon,African_crocodile,African_elephant,African_hunting_dog,American_Staffordshire_terrier,American_alligator,Appenzeller,Arabian_camel,Australian_terrier,...,wolf_spider,wombat,wooden_spoon,wool,worm_fence,wreck,yawl,yellow_lady's_slipper,yurt,zebra
id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X_train = objects.loc[movies.index.difference(movies_test.index)]

In [22]:
X_test = objects.loc[movies_test.index]

In [23]:
X_train

,Afghan_hound,African_chameleon,African_crocodile,African_elephant,African_hunting_dog,American_Staffordshire_terrier,American_alligator,Appenzeller,Arabian_camel,Australian_terrier,...,wolf_spider,wombat,wooden_spoon,wool,worm_fence,wreck,yawl,yellow_lady's_slipper,yurt,zebra
id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_test

,Afghan_hound,African_chameleon,African_crocodile,African_elephant,African_hunting_dog,American_Staffordshire_terrier,American_alligator,Appenzeller,Arabian_camel,Australian_terrier,...,wolf_spider,wombat,wooden_spoon,wool,worm_fence,wreck,yawl,yellow_lady's_slipper,yurt,zebra
id,,,,,,,,,,,,,,,,,,,,,
17335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
757,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
mlb_genres = MultiLabelBinarizer()

In [26]:
mlb_genres.fit(movies["genre_names"])

MultiLabelBinarizer()

In [27]:
genres = pandas.DataFrame(
    mlb_genres.transform(movies["genre_names"]),
    columns=mlb_genres.classes_,
    index=movies.index
)


In [28]:
genres

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
19996,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [29]:
Y_train = genres.loc[movies.index.difference(movies_test.index)]

In [30]:
Y_test = genres.loc[movies_test.index]

## Network Architecture

In [31]:
n_objects = objects.shape[1]
n_genres = genres.shape[1]

In [32]:
output_layer = keras.layers.Dense(
    units=n_genres,
    activation="sigmoid"
)

In [33]:
net = keras.models.Sequential(
    [
        keras.layers.Input(
            shape=n_objects
        ),
        keras.layers.Dense(
            units=128
        ),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(
            units=n_genres,
            activation="sigmoid"
        )
    ]
)

In [34]:
net.compile(
    loss="binary_crossentropy",
    optimizer="adam",
)

In [35]:
net.fit(
    X_train,
    Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[
        keras.callbacks.EarlyStopping(
            min_delta=0.00001,
            patience=8
        )
    ]
)

Epoch 1/100
347/347 [==============================] - 1s 3ms/step - loss: 0.4655 - val_loss: 0.3085
Epoch 2/100
347/347 [==============================] - 0s 1ms/step - loss: 0.3057 - val_loss: 0.3030
Epoch 3/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2974 - val_loss: 0.3013
Epoch 4/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2931 - val_loss: 0.3013
Epoch 5/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2901 - val_loss: 0.3016
Epoch 6/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2916 - val_loss: 0.3029
Epoch 7/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2863 - val_loss: 0.3037
Epoch 8/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2860 - val_loss: 0.3052
Epoch 9/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2848 - val_loss: 0.3062
Epoch 10/100
347/347 [==============================] - 0s 1ms/step - loss: 0.2839 - val_lo

In [36]:
Y_pred = pandas.DataFrame(
    net.predict(X_test).round(),
    columns=genres.columns,
    index=X_test.index
)


In [37]:
from sklearn.metrics import f1_score

In [38]:
f1_score(
    Y_test,
    Y_pred,
    average="micro"
)

0.22919508867667124

In [39]:
f1_score(
    Y_test,
    Y_pred,
    average="weighted"
)

0.18757989100504738

In [40]:
Y_pred

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
17335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11886,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1830,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
Y_test

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
17335,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
757,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0
11886,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2115,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1915,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18002,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1830,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rf_classifier = RandomForestClassifier()

In [44]:
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier()

In [45]:
Y_pred = rf_classifier.predict(X_test)

In [46]:
f1_score(
    Y_test,
    Y_pred,
    average="micro"
)

0.23249669749009244

In [47]:
f1_score(
    Y_test,
    Y_pred,
    average="weighted"
)

0.2023975315884658